In [49]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

print("CUDA verfügbar:", torch.cuda.is_available())
print("Torch CUDA Version:", torch.version.cuda)

torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

2.5.1+cu124
0.20.1+cu124
CUDA verfügbar: True
Torch CUDA Version: 12.4
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0

In [50]:
#!pip install pip==23.3
#!pip install -r /kaggle/input/piper-orig/piper-master/src/python/requirements.txt --force-reinstall
#!pip cache purge

#!pip uninstall --yes torchmetrics pytorch-lightning torch cython piper-phonemize  librosa numpy onnxruntime onnx onnxruntime-gpu matplotlib

!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.26 onnxruntime>=1.15.0 pytorch-lightning
!pip install --upgrade transformers
5
#!pip install torch==2.0.1 torchvision==0.15.2 --upgrade --force-reinstall

5

In [51]:
if not os.path.exists("/kaggle/input/pretrained"):
    raise Exception("upload pretrained model as pretrained.zip first. Models can be downloaded from: https://huggingface.co/rhasspy/piper-voices/tree/v1.0.0")
!cp -r /kaggle/input/pretrained /kaggle/working/pretrained

In [52]:
#@markdown # <font color="ffc800"> **Install software.** 📦
#@markdown ---
#@markdown ####In this cell the synthesizer and its necessary dependencies to execute the training will be installed. (this may take a while)


!cd /kaggle/working/
#!rm -r /kaggle/working/piper
# clone:
#!pip install -q -r requirements.txt
#!git clone -q https://github.com/rmcpantoja/piper
!git clone -q https://github.com/t-meinhardt/piper-master /kaggle/working/piper

!ls /kaggle/working/

!bash /kaggle/working/piper/src/python/build_monotonic_align.sh

!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"

# Useful vars:
use_whisper = True
print("Done!")

fatal: destination path '/kaggle/working/piper' already exists and is not an empty directory.
'=0.29.0'   audio_cache   piper_output	 resample.py
'=0.9.2'    packages	  pretrained	 voice-de_DE-epic-medium
'=1.15.0'   piper	  python
Compiling /kaggle/working/piper/src/python/piper_train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /kaggle/working/piper/src/python/piper_train/vits/monotonic_align/core.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /kaggle/working/piper/src/python/piper_train/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want 

In [54]:
# 1. Load existing dataset 📥
#preparation:

#@markdown ####**Important: the transcription means writing what the character says in each of the audios, and it must have the following structure:**

#@markdown ##### For a single-speaker dataset:
#@markdown * wavs/1.wav|This is what my character says in audio 1.
#@markdown * wavs/2.wav|This, the text that the character says in audio 2.
#@markdown * ...

#@markdown ##### For a multi-speaker dataset:

#@markdown * wavs/speaker1audio1.wav|speaker1|This is what the first speaker says.
#@markdown * wavs/speaker1audio2.wav|speaker1|This is another audio of the first speaker.
#@markdown * wavs/speaker2audio1.wav|speaker2|This is what the second speaker says in the first audio.
#@markdown * wavs/speaker2audio2.wav|speaker2|This is another audio of the second speaker.
#@markdown * ...

#@markdown And so on. In addition, the transcript must be in a **.csv format. (UTF-8 without BOM). Save it as metadata.csv outside of folder "wavs"**

import os
import wave
import datetime

def get_dataset_duration(wav_path):
    totalduration = 0
    for file_name in [x for x in os.listdir(wav_path) if os.path.isfile(os.path.join(wav_path, x)) and ".wav" in x]:
        with wave.open(os.path.join(wav_path, file_name), "rb") as wave_file:
            frames = wave_file.getnframes()
            rate = wave_file.getframerate()
            duration = frames / float(rate)
            totalduration += duration
    wav_count = len([x for x in os.listdir(wav_path) if x.endswith('.wav')])
    duration_str = str(datetime.timedelta(seconds=round(totalduration, 0)))
    return wav_count, duration_str

sample_path = '/kaggle/input/samples/WAVs'
wavs_dir = os.path.join(sample_path, "wavs")  


if not os.path.exists(wavs_dir):
    raise Exception(f"WAV directory not found at: {wavs_dir}. Upload WAV files as samples.zip first.")

audio_count, dataset_dur = get_dataset_duration(wavs_dir)
print(f"Opened dataset with {audio_count} wavs with duration {dataset_dur}.")


Opened dataset with 23 wavs with duration 0:08:59.


In [58]:
print("Adapt parameters here")
language = "Deutsch" 
model_name = "epic_german" 
dataset_format = "ljspeech"  
single_speaker = True
force_sp = " --single-speaker" if single_speaker else ""
sample_rate = "16000"

checkpoint_name_pretrained_model = "epoch=6069-step=230660.ckpt"

# === Training Configuration ===

# === Batch size ===
# Number of training examples per step.
# Higher = faster learning, but uses more memory.
batch_size = 4  # e.g., 16

# === Floating-point precision ===
# 16-mixed = mixed precision (faster, uses less RAM), 32 = full precision.
# Recommended: 16-mixed for GPUs.
precision = "16-mixed"

# === Model quality ===
# Affects model size, training time, and audio resolution.
# Options:
# - x-low: 16kHz audio, 5–7M parameters
# - medium: 22.05kHz audio, ~15–20M parameters
# - high: 22.05kHz audio, 28–32M parameters
quality = "medium"

# === Checkpoint saving interval (in epochs) ===
# How often to save a model checkpoint during training.
# 1 = save after every epoch.
checkpoint_epochs = 1

# === Number of best model checkpoints to keep ===
# Keeps the most recent `num_ckpt` best models based on validation.
# Set to 0 to disable saving multiple checkpoints.
num_ckpt = 2

# === Save the final model ===
# Save a "last.ckpt" file at the end of training.
# If num_ckpt = 0, this will be the only saved model.
# ⚠️ Not recommended for very small datasets (may result in a corrupt checkpoint).
save_last = True

# === Logging interval ===
# How often to log training progress (loss, metrics).
# Lower = more detailed logs.
log_every_n_steps = 1000

# === Max training epochs ===
# Total number of full passes through the training dataset.
# Use fewer epochs for small datasets.
max_epochs = 10

# === Finetuning ===
# Optional: if you want to continue training from a pre-trained model,
# uncomment and set the path to the checkpoint file (.ckpt).
# ft_command = "--finetune path/to/model.ckpt"

Adapt parameters here


In [59]:

import os

use_whisper = False
if use_whisper:
    import torch
    from faster_whisper import WhisperModel
    from tqdm import tqdm

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    def make_dataset(path, language):
        metadata = ""
        text = ""
        files = [f for f in os.listdir(path) if f.endswith(".wav")]
        assert len(files) > 0, "No WAV files found. Please check your path."
        
        metadata_file = open(f"{path}/../metadata.csv", "w", encoding='utf-8')
        whisper = WhisperModel("large-v3", device=device, compute_type="float16")
        for audio_file in tqdm(files):
            full_path = os.path.join(path, audio_file)
            segments, _ = whisper.transcribe(full_path, word_timestamps=False, language=language)
            for segment in segments:
                text += segment.text
            text = text.strip().replace('\n', ' ')
            metadata_line = f"{audio_file}|{text}\n"
            metadata_file.write(metadata_line)
            text = ""
        metadata_file.close()
        del whisper
        print("Transcription and metadata creation done.")


languages = {
    "ألعَرَبِي": "ar",
    "Català": "ca",
    "čeština": "cs",
    "Dansk": "da",
    "Deutsch": "de",
    "Ελληνικά": "el",
    "English (British)": "en",
    "English (U.S.)": "en-us",
    "Español (Castellano)": "es",
    "Español (Latinoamericano)": "es-419",
    "Suomi": "fi",
    "Français": "fr",
    "Magyar": "hu",
    "Icelandic": "is",
    "Italiano": "it",
    "ქართული": "ka",
    "қазақша": "kk",
    "Lëtzebuergesch": "lb",
    "नेपाली": "ne",
    "Nederlands": "nl",
    "Norsk": "nb",
    "Polski": "pl",
    "Português (Brasil)": "pt-br",
    "Português (Portugal)": "pt-pt",
    "Română": "ro",
    "Русский": "ru",
    "Српски": "sr",
    "Svenska": "sv",
    "Kiswahili": "sw",
    "Türkçe": "tr",
    "украї́нська": "uk",
    "Tiếng Việt": "vi",
    "简体中文": "zh"
}

def _get_language(code):
    return languages[code]

final_language = _get_language(language)

work_dir = '/kaggle/working'
output_path = os.path.join(work_dir, 'piper_output') 
output_dir = os.path.join(output_path, model_name)

os.makedirs(output_dir, exist_ok=True)

audio_cache_dir = os.path.join(work_dir, 'audio_cache')
os.makedirs(audio_cache_dir, exist_ok=True)

# Resampling 
resample = False
if resample:
    input_wavs_dir = wavs_dir  
    output_wavs_dir = '/kaggle/working/wavs_resampled'

    os.makedirs(output_wavs_dir, exist_ok=True)
    !python /kaggle/working/python/resample.py \
        --input_dir "{input_wavs_dir}" \
        --output_dir "{output_wavs_dir}" \
        --output_sr {sample_rate} \
        --file_ext "wav"

if use_whisper:
    print("Transcript file missing. Generating transcripts with Whisper...")
    make_dataset(f"{wavs_dir}", final_language[:2])
    print("Transcription done!")

print("Starting preprocessing...")
%cd /kaggle/working/python
!python -m piper_train.preprocess \
  --language {final_language} \
  --input-dir "{sample_path}" \
  --cache-dir "{audio_cache_dir}" \
  --output-dir "{output_dir}" \
  --dataset-name "{model_name}" \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

#print("ls /kaggle/working/piper_output/epic")
#!ls /kaggle/working/piper_output/epic
#print("ls /kaggle/working/audio_cache")
#!ls /kaggle/working/audio_cache
#print("cat config")
#!cat /kaggle/working/piper_output/epic/config.json
print("Preprocessing done!")


Starting preprocessing...
/kaggle/working/python
Preprocessing done!


In [60]:
#@markdown # <font color="ffc800"> **4. Settings.** 🧰
#@markdown ---
import json
from IPython.display import display
import os
import re
import glob
import json
import os
import gdown
#@markdown ### <font color="orange">**Select the action to train this dataset: (READ CAREFULLY)**

#@markdown * The option to <font color="orange">continue a training</font> is self-explanatory. If you've previously trained a model with free colab, your time is up and you're considering training it some more, this is ideal for you. You just have to set the same settings that you set when you first trained this model.
#@markdown * The option to <font color="orange">convert a single-speaker model to a multi-speaker model</font> is self-explanatory, and for this it is important that you have processed a dataset that contains text and audio from all possible speakers that you want to train in your model.
#@markdown * The <font color="orange">finetune</font> option is used to train a dataset using a pretrained model, that is, train on that data. This option is ideal if you want to train a very small dataset (more than five minutes recommended).
#@markdown * The <font color="orange">train from scratch</font> option builds features such as dictionary and speech form from scratch, and this may take longer to converge. For this, hours of audio (8 at least) are recommended, which have a large collection of phonemes.

action = "finetune" #@param ["Continue training", "convert single-speaker to multi-speaker model", "finetune", "train from scratch"]
#@markdown ---
if action == "Continue training":
    checkpoints = glob.glob(f"{output_dir}/lightning_logs/**/checkpoints/last.ckpt", recursive=True)
    if len(checkpoints):
        last_checkpoint = sorted(checkpoints, key=lambda x: int(re.findall(r'version_(\d+)', x)[0]))[-1]
        ft_command = f'--resume_from_checkpoint "{last_checkpoint}" '
        print(f"Continuing {model_name}'s training at: {last_checkpoint}")
    else:
        raise Exception("Training cannot be continued as there is no checkpoint to continue at.")
elif action == "finetune":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        raise Exception("Oh no! You have already trained this model before, you cannot choose this option since your progress will be lost, and then your previous time will not count. Please select the option to continue a training.")
    else:
        ft_command = f"--resume_from_checkpoint /kaggle/working/pretrained/{checkpoint_name_pretrained_model} "
elif action == "convert single-speaker to multi-speaker model":
    if not single_speaker:
        ft_command = f"--resume_from_single_speaker_checkpoint /kaggle/working/pretrained/{checkpoint_name_pretrained_model} "
    else:
        raise Exception("This dataset is not a multi-speaker dataset!")
else:
    ft_command = ""
if action == "convert single-speaker to multi-speaker model" or action == "finetune":
    if os.path.exists(f"/kaggle/working/pretrained/{checkpoint_name_pretrained_model}"):
        print("\033[93mModel found!")
    else:
        raise Exception("No Pretrained Model!")
        
    #try:
    #    with open('/kaggle/input/piper-orig/piper-master/notebooks/pretrained_models.json') as f:
    #        pretrained_models = json.load(f)
    #    
    #    if final_language in pretrained_models:
    #        models = pretrained_models[final_language]
    #        model_names = list(models.keys())  # Liste der Modellnamen
    #        print(f"Verfügbare Modelle: {model_names}")


            
            # Anstatt ein Dropdown-Menü zu verwenden, wähle ein Modell per Eingabe
            #model_name = input("Bitte wähle ein Modell aus der Liste (z.B. 'epic'): ")
            #if model_name in models:
            #    model_url = models[model_name]
            #    print(f"\033[93mModel {model_name} wird heruntergeladen...")
            #    if model_url.startswith("1"):
            #        gdown.download(model_url, "/kaggle/working/pretrained.ckpt", quiet=False)
            #    elif model_url.startswith("https://drive.google.com/file/d/"):
            #        gdown.download(model_url, "/kaggle/working/pretrained.ckpt", quiet=False, fuzzy=True)
            #    else:
            #        os.system(f"wget --quiet {model_url} -O /kaggle/working/pretrainted_model/pretrained.ckpt")
                
            #    if os.path.exists("/kaggle/working/pretrained.ckpt"):
            #        print("\033[93mModel heruntergeladen!")
            #    else:
            #        raise Exception("Das Pretrained-Modell konnte nicht heruntergeladen werden!")
            #else:
            #    raise Exception(f"Das Modell {model_name} existiert nicht in der Liste.")
    #    else:
    #        raise Exception(f"Es gibt keine vortrainierten Modelle für die Sprache {final_language}")
    #except FileNotFoundError:
    #    raise Exception("Die Datei pretrained_models.json wurde nicht gefunden.")

else:
    print("\033[93mWarning: this model will be trained from scratch. You need at least 8 hours of data for everything to work decent. Good luck!")




Model found!


In [ ]:
#@markdown # <font color="ffc800"> **6. Train.** 🏋️‍♂️
#@markdown ---
#@markdown ### Run this cell to train your final model!

#@markdown ---
#@markdown ### <font color="orange">**Disable validation?**
#@markdown By uncheck this checkbox, this will allow to train the full dataset, without using any audio files or examples as a validation set. So, it will not be able to generate audios on the tensorboard while it's training. It is recommended to disable validation on extremely small datasets.

validation = True #@param {type:"boolean"}
if validation:
    validation_split = 0.01
    num_test_examples = 1
else:
    validation_split = 0
    num_test_examples = 0

if not save_last:
    save_last_command = ""
else:
    save_last_command = "--save_last True "
%ls /kaggle/working/
%cd /kaggle/working/python
# Ensure the training command uses paths that exist in the Kaggle environment
get_ipython().system(f'''
python -m piper_train \
--dataset-dir {output_dir} \
--accelerator 'gpu' \
--devices 1 \
--batch-size {batch_size} \
--validation-split {validation_split} \
--num-test-examples {num_test_examples} \
--quality {quality} \
--checkpoint-epochs {checkpoint_epochs} \
--num_ckpt {num_ckpt} \
{save_last_command}\
--log_every_n_steps {log_every_n_steps} \
--max_epochs {max_epochs} \
{ft_command}\
--precision {precision}
''')

print("---------------------------------------------------------------------------------------------------------------------------------------")
print(f"training finished! saved to {output_dir}")
!ls /kaggle/working/piper_output/epic

In [ ]:
#when training is finished model needs to be exported to onnx
#model files are saved in the specified working folder
%ls /kaggle/working/piper_output/epic/lightning_logs/
%ls /kaggle/working/piper_output/epic/lightning_logs/version_0/checkpoints

In [ ]:
import os
import json
import subprocess

print("export ckpt + json file from training to ONNX...")

%ls /kaggle/working/
%cd /kaggle/working/python
# Eingaben:
#model_url = "https://..."  # Direktlink zum .ckpt-Modell
#config_url = "https://..."  # Direktlink zur .json-Konfig

language = "de_DE"
voice_name = "epic"
quality = "medium"
streaming = False

output_path = "/kaggle/working/piper_output/epic"  # vom Training erzeugt
ckpt_path = os.path.join(output_path, "lightning_logs/version_0/checkpoints")
#ckpt_path = os.path.join(output_path, "lightning_logs/version_3/checkpoints")
model_path = os.path.join(ckpt_path, "last.ckpt")  # oder ändere, wenn du epoch=xxx.ckpt willst
config_path = os.path.join(output_path, "config.json")

export_voice_name = f"{language}-{voice_name}+RT-{quality}" if streaming else f"{language}-{voice_name}-{quality}"
export_voice_path = f"/kaggle/working/voice-{export_voice_name}"
packages_path = "/kaggle/working/packages"
#model_path = "/kaggle/working/model.ckpt"
#config_path = f"{export_voice_path}/{export_voice_name}.onnx.json"

os.makedirs(export_voice_path, exist_ok=True)
os.makedirs(packages_path, exist_ok=True)

#subprocess.run(["wget", model_url, "-O", model_path])
#subprocess.run(["wget", config_url, "-O", config_path])

config_export_path = os.path.join(export_voice_path, f"{export_voice_name}.onnx.json")
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)

if streaming:
    config["streaming"] = True
    config["key"] = export_voice_name

with open(config_export_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4)

# Exportieren
if streaming:
    subprocess.run([
        "python", "-m", "piper_train.export_onnx_streaming",
        model_path, export_voice_path
    ])
else:
    subprocess.run([
        "python", "-m", "piper_train.export_onnx",
        model_path, f"{export_voice_path}/{export_voice_name}.onnx"
    ])


# Paket erstellen
subprocess.run([
    "tar", "-czvf",
    f"{packages_path}/{export_voice_name}.tar.gz",
    "-C", export_voice_path, "."
])

print(f"finshied: file = {packages_path}/{export_voice_name}.tar.gz")

In [ ]:
if not os.path.exists("/kaggle/input/piper-tts"):
    raise Exception("upload piper tts binary piper_arch.tar.gz (e.g. piper_amd64.tar.gz) as piper-tts first")
!cp -r /kaggle/input/piper-tts /kaggle/working/

In [ ]:
import subprocess
import os

model_path = f"{export_voice_path}/{export_voice_name}.onnx"

text = "this is a test!"

work_dir = '/kaggle/working'
output_dir = os.path.join(work_dir, 'tts-test')  
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "test.wav")

# Schritt 5: Weitere Einstellungen
dataset_format = "ljspeech"  # Formattyp
single_speaker = True
force_sp = " --single-speaker" if single_speaker else ""
sample_rate = "16000"

# Audio-Cache erstellen
audio_cache_dir = os.path.join(work_dir, 'audio_cache')
os.makedirs(audio_cache_dir, exist_ok=True)

# Shell-Befehl mit Text über Pipe weitergeben
subprocess.run(
    f"echo '{text}' | /kaggle/working/piper-tts/piper --model {model_path} --output_file {output_file}",
    shell=True,
    check=True
)
